In [7]:
import requests
from dotenv import load_dotenv
import os 
import pandas as pd
from newsapi import NewsApiClient
import alpaca_trade_api as tradeapi
import json
import hvplot.pandas
from MCForecastTools import MCSimulation

In [8]:
load_dotenv()

True

In [9]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpha_vantage_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [10]:
stock_tickers = ["AMZN", "GOOG", "TSLA", "MCD", "XOM"] 
crypto_tickers = ["BTC/USD", "SOL/USD", "LTC/USD", "DOGE/USD", "ETH/USD"]
start_date = pd.Timestamp("2018-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-05-01", tz="America/New_York").isoformat()
timeframe = "1Day"

In [79]:
stock_df_portfolio = alpaca.get_bars(
    stock_tickers,
    timeframe,
    start_date,
    end_date
).df

stock_df_portfolio.index = stock_df_portfolio.index.date
stock_df_portfolio.drop(columns=['volume', 'open', 'high', 'low','vwap','trade_count'], inplace=True)

In [80]:
amzn = stock_df_portfolio[stock_df_portfolio["symbol"]=="AMZN"].drop("symbol", axis=1)
goog = stock_df_portfolio[stock_df_portfolio["symbol"]=="GOOG"].drop("symbol", axis=1)
tsla = stock_df_portfolio[stock_df_portfolio["symbol"]=="TSLA"].drop("symbol", axis=1)
mcd = stock_df_portfolio[stock_df_portfolio["symbol"]=="MCD"].drop("symbol", axis=1)
xom = stock_df_portfolio[stock_df_portfolio["symbol"]=="XOM"].drop("symbol", axis=1)

stock_df_portfolio = pd.concat([amzn, goog, tsla, mcd, xom], axis=1, keys=["AMZN","GOOG","TSLA","MCD","XOM"])
stock_df_portfolio.head()


,AMZN,GOOG,TSLA,MCD,XOM
,close,close,close,close,close
2018-05-01,1582.260,1037.31,299.92,163.44,76.95
2018-05-02,1569.680,1024.38,301.15,160.68,76.80
2018-05-03,1572.075,1023.72,284.45,160.08,76.54
2018-05-04,1580.950,1048.21,294.09,165.03,76.90
2018-05-07,1600.140,1054.79,302.77,165.00,77.74


In [45]:
amzn_close = stock_df_portfolio['AMZN']['close']#.hvplot(label='Amazon')
goog_close = stock_df_portfolio['GOOG']['close']#.hvplot(label='Google')
tsla_close = stock_df_portfolio['TSLA']['close']#.hvplot(label='Tesla')
mcd_close = stock_df_portfolio['MCD']['close']#.hvplot(label="McDonald's")
xom_close = stock_df_portfolio['XOM']['close']#.hvplot(label="Exxon Mobil")

In [126]:
stock_df_portfolio_close = pd.concat([amzn_close, goog_close, tsla_close, mcd_close, xom_close], axis=1, keys=["AMZN","GOOG","TSLA","MCD","XOM"])
stock_close_plot = stock_df_portfolio_close.hvplot(ylabel='Price',xlabel='Time',title='Stock Portfolio Close Price')


In [47]:
crypto_df_portfolio = alpaca.get_crypto_bars(
    crypto_tickers,
    timeframe,
    start_date,
    end_date
).df

crypto_df_portfolio.index = crypto_df_portfolio.index.date
crypto_df_portfolio.drop(columns=['volume', 'open', 'high', 'low','vwap','trade_count'], inplace=True)

In [48]:
btc = crypto_df_portfolio[crypto_df_portfolio["symbol"]=="BTC/USD"].drop("symbol", axis=1)
sol = crypto_df_portfolio[crypto_df_portfolio["symbol"]=="SOL/USD"].drop("symbol", axis=1)
ltc = crypto_df_portfolio[crypto_df_portfolio["symbol"]=="LTC/USD"].drop("symbol", axis=1)
doge = crypto_df_portfolio[crypto_df_portfolio["symbol"]=="DOGE/USD"].drop("symbol", axis=1)
eth = crypto_df_portfolio[crypto_df_portfolio["symbol"]=="ETH/USD"].drop("symbol", axis=1)

crypto_df_portfolio = pd.concat([btc, sol, ltc, doge, eth], axis=1, keys=["BTC","SOL","LTC","DOGE","ETH"])
crypto_df_portfolio 

,BTC,SOL,LTC,DOGE,ETH
,close,close,close,close,close
2021-01-01,29676.79,1.8036,126.07,0.0073,733.65
2021-01-02,33769.52,1.9576,137.41,0.0121,786.88
2021-01-03,32908.02,2.3033,165.00,0.0101,1086.86
2021-01-04,30441.57,2.1059,147.78,0.0091,991.40
2021-01-05,35063.00,2.0000,161.47,0.0104,1105.56
...,...,...,...,...,...
2023-04-26,29119.00,21.9100,89.25,0.0803,1911.79
2023-04-27,29537.16,22.4300,89.51,0.0804,1916.09
2023-04-28,29420.00,23.3900,90.15,0.0812,1904.49


In [49]:
btc_close = crypto_df_portfolio["BTC"]['close']#.hvplot(label='Bitcoin')
sol_close = crypto_df_portfolio["SOL"]['close']#.hvplot(label='Solana')
ltc_close = crypto_df_portfolio["LTC"]['close']#.hvplot(label='Litecoin')
doge_close = crypto_df_portfolio["DOGE"]['close']#.hvplot(label='Doge Coin')
eth_close = crypto_df_portfolio["ETH"]['close']#.hvplot(label='Ethereum')

In [125]:
crypto_df_portfolio_close = pd.concat([btc_close, sol_close, ltc_close, doge_close, eth_close], axis=1, keys=["BTC","SOL","LTC","DOGE","ETH"])
crypto_close_plot = crypto_df_portfolio_close.hvplot(ylabel='Price',xlabel='Time',title='Crypto Portfolio Close Price')

In [76]:
complete_df_portfolio = pd.concat([stock_df_portfolio,crypto_df_portfolio])
#complete_df_portfolio.columns = ['AMZN','BTC','DOGE','ETH','GOOG','LTC','MCD','SOL','TSLA','XOM']
complete_df_portfolio

,AMZN,BTC,DOGE,ETH,GOOG,LTC,MCD,SOL,TSLA,XOM
,close,close,close,close,close,close,close,close,close,close
2018-05-01,1582.260,NaN,NaN,NaN,1037.31,NaN,163.44,NaN,299.92,76.95
2018-05-02,1569.680,NaN,NaN,NaN,1024.38,NaN,160.68,NaN,301.15,76.80
2018-05-03,1572.075,NaN,NaN,NaN,1023.72,NaN,160.08,NaN,284.45,76.54
2018-05-04,1580.950,NaN,NaN,NaN,1048.21,NaN,165.03,NaN,294.09,76.90
2018-05-07,1600.140,NaN,NaN,NaN,1054.79,NaN,165.00,NaN,302.77,77.74
...,...,...,...,...,...,...,...,...,...,...
2023-04-26,NaN,29119.00,0.0803,1911.79,NaN,89.25,NaN,21.91,NaN,NaN
2023-04-27,NaN,29537.16,0.0804,1916.09,NaN,89.51,NaN,22.43,NaN,NaN
2023-04-28,NaN,29420.00,0.0812,1904.49,NaN,90.15,NaN,23.39,NaN,NaN


In [123]:
stock_daily_returns = stock_df_portfolio.pct_change()
weights = [.2,.2,.2,.2,.2]

financefolio_stock_returns = stock_daily_returns.dot(weights)
financefolio_stock_plot = financefolio_stock_returns.hvplot(label='Financefolio Stocks Daily Returns') 

financefolio_stock_plot


:Curve   [index]   (0)

In [127]:
crypto_daily_returns = crypto_df_portfolio.pct_change()
weights = [.2,.2,.2,.2,.2]

financefolio_crypto_returns = crypto_df_portfolio.dot(weights)
financefolio_crypto_plot = financefolio_crypto_returns.hvplot(label='Financefolio Crypto Daily returns') 

financefolio_crypto_plot

:Curve   [index]   (0)

In [128]:
crypto_close_plot

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [129]:
stock_close_plot

:NdOverlay   [Variable]
   :Curve   [index]   (value)